# Working with a database

This week's assignment has a few basic steps.  First, we're going to pull some data down off the internet and store it into our MySQL database.  Make sure that you use your username as part of the table name as show in the examples so that you don't create a problem for other students.

Then, we'll merge that with some data already in the database and calculate a few results.  When it comes to calculating the results, you can do so either with SQL or with Pandas operations.


## PART 1: Setup your database connection and table name

In the code below, change the value of the variable `MYTABLE` to use your own username rather that `'pboal'`

You can then use `MYTABLE` in the rest of your code to reference that table name.

In [29]:
#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!
# First, put in your user name below
#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!
import os
import pymysql
from sqlalchemy import create_engine
import pandas as pd
host = 'slucor2020.cgdcoitnku0k.us-east-1.rds.amazonaws.com'
port = '3306'
user = 'slucor2020'
password = 'SLUcor2020'
database = 'hds5210'

MYTABLE='kenny0415' + '_data'


# Then, create a connection to the same database, using the 
# same user name and password in the `mysql-examples.ipynb` 
# file.  Use conn as the database connection variable.

conn =  create_engine('mysql+pymysql://' + 
                     user + ':' + 
                     password + '@' + 
                     host + '/' + 
                     database, echo=False)


In [30]:
assert(MYTABLE != 'paulboal_data')
assert(conn.connect())

## PART 2: Bring in outside data

Grab the data from this URL and put it into a database table named with your `username_data`.

http://dhcs-chhsagency.opendata.arcgis.com/datasets/8e4f3a0c75b9424d888d11c1f949cc32_0.csv

By the end of your cell, the table should be created.  The tests are going to verify that the table exists and looks right.

In [40]:
url = 'http://dhcs-chhsagency.opendata.arcgis.com/datasets/8e4f3a0c75b9424d888d11c1f949cc32_0.csv'
df=pd.read_csv(url)
df.to_sql(MYTABLE, conn, index=False)

### SOLUTION GOES HERE


In [41]:
dxyz = pd.read_sql_query('SELECT * FROM ' + MYTABLE, conn)
dxyz.shape

(328, 19)

In [42]:
dxyz = pd.read_sql_query('SELECT * FROM ' + MYTABLE, conn)
assert(dxyz.shape == (328,19))
assert(list(dxyz.columns) == ['Provider_Name', 'NPI', 'CCN',
       'Medicaid_EP_Hospital_Type', 'Street_Address', 'City', 'County',
       'State', 'Zip_Code', 'Payment_Year_Number', 'Program_Type',
       'Total_payments', 'Last_Program_Year', 'Last_Payment_Year',
       'Last_Payment_Criteria', 'Most_Recent_Disbursement_Amount', 'LONGITUDE',
       'LATITUDE', 'FID'])

In [39]:
#with conn.connect() as c:
     #c.execute('DROP TABLE IF EXISTS ' +MYTABLE)

## PART 3: Combine with other data in the database

In the database, there is an existing table called `population`.  We want to merge the DHCS datafile loaded above with the population data available in this other database table  The tables can be merged on `MYTABLE`'s `Zip_Code` field and `population`'s `zipcode` field.

Note that not all `Zip_Codes` from your downloaded file have to be in the `population` table.  If they aren't, then I want you to eliminate the non-matching records.  That is, only keep the records that have a matching ZIP code in both sets of data.

Answer the question:
Which providers are located in the zipcode with the largest population?

Put your answer in the form `answer = ['a', 'list', 'of', 'NPI', 'like', '1593042103]`

In [52]:
pd.read_sql_query('SHOW TABLES', conn)

,Tables_in_hds5210
0,NPI
1,Provider_Name
2,Test
3,aavila_data
4,aavila_data_NPI
5,aavila_data_new
6,aavila_data_year
7,aavila_datax
8,abbieslu
9,abbieslu_NPI


In [72]:
### SOLUTION GOES HERE
answer=pd.read_sql_query("""
  SELECT k.NPI
  FROM 
    population p INNER JOIN
    kenny0415_data k ON k.Zip_Code = p.zipcode
    ORDER BY 
    p.population desc limit 1;
""", conn)
#answer=answer.astype(str).values.tolist()[0]
list(answer['NPI'].astype(str))

['1194016923']

In [45]:
pd.read_sql_query("SELECT * FROM population", conn)

,zipcode,population,median_age,total_males,total_females,total_households,avg_household
0,91371,1,73.5,0,1,1,1.00
1,90001,57110,26.6,28468,28642,12971,4.40
2,90002,51223,25.5,24876,26347,11731,4.36
3,90003,66266,26.3,32631,33635,15642,4.22
4,90004,62180,34.8,31302,30878,22547,2.73
...,...,...,...,...,...,...,...
314,93552,38158,28.4,18711,19447,9690,3.93
315,93553,2138,43.3,1121,1017,816,2.62
316,93560,18910,32.4,9491,9419,6469,2.92
317,93563,388,44.5,263,125,103,2.53


In [46]:
assert(type(answer) == list)
assert(answer == ['1194016923'])

## PART 4: Total by hospital type

This is a multistep process:
* From our downloaded data file, compute the `Total payments` per ZIP code and Medicaid EP Hospital Type.
* Then merge that with the `population` data to compute a `Total payments` per person.
* Then average that across all of the `Medicaid EP Hospital Types` to get an average per persona payment for these type of hospital.

Your answer should be in structure of a data frame with at least two columns:
* Medicaid_EP_Hospital_Type
* Avg_Pay_per_Capita

In [47]:
tables = {}
for n,t in pd.read_sql_query("SHOW TABLES", conn).iterrows():
    name = str(t['Tables_in_hds5210'])
    print(name)
    tables[name] = pd.read_sql("SELECT * FROM "+name,conn)

NPI
Provider_Name
Test
aavila_data
aavila_data_NPI
aavila_data_new
aavila_data_year
aavila_datax
abbieslu
abbieslu_NPI
abbieslu_ipv
abbieslu_npi
abbieslu_year
apuryear_data
arefehesmaeilpour_data
ashley_data
btabman_data
corona_counts
county_population
ed_visits
hospital_services
hospitals
jdgendron_data
jmccar23_data
kaylaschmidt_data
kenny0415_data
khalafh_data
liannecota_data
paulboal_data
population
sam
sampele
system_affiliations
x


In [48]:
### SOLUTION GOES HERE
tables['kenny0415_data']['Zip_Code']=tables['kenny0415_data']['Zip_Code'].astype(str)
kenny0415_data_groupby=tables['kenny0415_data'].groupby(["Zip_Code","Medicaid_EP_Hospital_Type"]).sum().reset_index()
tables['population']['zipcode']=tables['population']['zipcode'].astype(str)
merge=kenny0415_data_groupby.merge(tables['population'], how='inner', left_on='Zip_Code', right_on='zipcode')
merge['Total_payments_per_person']=merge['Total_payments'] / merge['population']
merge=merge.groupby('Medicaid_EP_Hospital_Type').mean()
merge['Avg_Pay_per_Capita']=merge['Total_payments_per_person']
answer = merge
answer

,NPI,CCN,Payment_Year_Number,Total_payments,Last_Program_Year,Last_Payment_Year,Most_Recent_Disbursement_Amount,LONGITUDE,LATITUDE,FID,population,median_age,total_males,total_females,total_households,avg_household,Total_payments_per_person,Avg_Pay_per_Capita
Medicaid_EP_Hospital_Type,,,,,,,,,,,,,,,,,,
Acute Care Hospitals,1.693329e+09,56481.066667,3.186667,3.245626e+06,2256.213333,2257.533333,760936.680000,-132.40065,38.128570,182.4,38896.720000,35.553333,19198.586667,19698.133333,12600.040000,2.969333,17974.362790,17974.362790
Children's Hospitals,1.370193e+09,53307.000000,2.666667,4.847082e+06,2014.666667,2015.666667,592707.333333,-118.25442,33.991295,203.0,42172.333333,34.400000,20820.000000,21352.333333,16875.666667,2.593333,111.559021,111.559021


In [50]:
assert(type(answer) == pd.core.frame.DataFrame)
assert(round(answer.query("Medicaid_EP_Hospital_Type == 'Acute Care Hospitals'")['Avg_Pay_per_Capita'].sum(),3) == 17974.363)
assert(round(answer.query("Medicaid_EP_Hospital_Type == 'Children\\'s Hospitals'")['Avg_Pay_per_Capita'].sum(),3) == 111.559)
